In [ ]:
import scipy.stats as sps
from sklearn.model_selection import train_test_split
import gc
from sklearn.linear_model import LinearRegression
from dateutil.parser import *
import pandas as pd
import numpy as np
import os
import yt.wrapper as yt
from collections import Counter
import json
import typing as tp
import ast
from datetime import datetime
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import operator
import re
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/')

In [ ]:
leads_last = sorted(lib.find_tables_in_hahn_folder("//home/cloud_analytics/dwh/raw/crm/leads"))[-1]
leads_last

# 1. Добавление

In [ ]:
req = f"""
SELECT
    task_id,
    first_name,
    last_name,
    account_name,
    phone_mobile,
    email,
    status,
    lead_source,
    lead_source_description,
    description,
    timezone,
    website
FROM (
    select DISTINCT
        lead_id as task_id, 
        any(first_name) as first_name,
        any(last_name) as last_name,
        any(account_name) as account_name, 
        any(phone_mobile) as phone_mobile,
        any(email) as email,
        status,
        lead_source,
        any(lead_source_description) as lead_source_description,
        any(description) as description,
        any(timezone) as timezone
    from 
        "//home/cloud_analytics/kulaga/leads_cube"
    where (lead_source = 'var'
    OR lead_source = 'referrals')
    and status in ('New', 'Assigned')
    GROUP BY task_id, status, lead_source
) as a
LEFT JOIN (
    SELECT
        DISTINCT
        id as lead_id,
        max(lower(website)) as website
    FROM "{leads_last}"
    GROUP BY lead_id
) as b
ON a.task_id == b.lead_id
WHERE phone_mobile != ''
AND isNotNull(phone_mobile)
FORMAT TabSeparatedWithNames
"""
df = lib.execute_query(req)

In [ ]:
isv_df = lib.execute_query(f"""
SELECT
    DISTINCT
        phone_mobile,
        email,
        website
FROM "//home/cloud_analytics/kulaga/leads_cube" as a
LEFT JOIN (
    SELECT
        DISTINCT
        id as lead_id,
        max(lower(website)) as website
    FROM "{leads_last}"
    GROUP BY lead_id
) as b
ON a.lead_id == b.lead_id
WHERE lead_source = 'isv'
AND phone_mobile != ''
AND isNotNull(phone_mobile)
FORMAT TabSeparatedWithNames
""")

In [ ]:
old_df = lib.execute_query("""
SELECT 
    DISTINCT
    task_id
FROM "//home/cloud_analytics/lunin-dv/call_center_output_streaming_output/var_source_drop_task_id"
FORMAT TabSeparatedWithNames
""")
df = df[~df["task_id"].isin(old_df["task_id"])]

In [ ]:
isv_df['phone_mobile'] = isv_df['phone_mobile'].apply(lambda x: lib.works_with_phones(x))
df['phone_mobile'] = df['phone_mobile'].apply(lambda x: lib.works_with_phones(x))

repeat_isv = df[(df['phone_mobile'].isin(isv_df['phone_mobile'].fillna('trash'))) |
                (df['email'].isin(isv_df['email'].fillna('trash'))) |
                (df['website'].isin(isv_df['website'].fillna('trash')))]

In [ ]:
df = df[~df["task_id"].isin(repeat_isv["task_id"])]

In [ ]:
str(datetime.now()).replace(" ", "T")

In [ ]:
df.drop(columns=['website'], inplace=True)

In [ ]:
if len(df) > 0:
    lib.save_table(str(datetime.now()).replace(" ", "T"), 
               "//home/cloud_analytics/call_center/input/var", df)

## dropped list update

In [ ]:
added_df = pd.DataFrame(df["task_id"])

In [ ]:
lib.save_table("var_source_drop_task_id", 
               "//home/cloud_analytics/lunin-dv/call_center_output_streaming_output",
               added_df, append=True)

# 1.25 Дубликат ISV

In [ ]:
double_isv_var_df = pd.DataFrame()
double_isv_var_df['CRM_Lead_ID'] = repeat_isv['task_id']
double_isv_var_df['Tags'] = '["дубль заявки isv/var"]'
double_isv_var_df['Assigned_to'] = 'nypetrova'

In [ ]:
dt = lib.execute_query("""
SELECT
    toInt64(NOW()) as Timestamp
FORMAT TabSeparatedWithNames
""")['Timestamp'].iloc[0]


In [ ]:
double_isv_var_df['Timestamp'] = dt
double_isv_var_df

In [ ]:
lib.save_table('update_leads', "//home/cloud_analytics/export/crm/update_call_center_leads",
               double_isv_var_df, append=True)

# 2. Update Lead

In [ ]:
req = """
SELECT
    toInt64(NOW()) as Timestamp,
    CAST(task_id as String) as CRM_Lead_ID,
    Billing_account_id as Billing_account_id_New,
    billing_account_id as Billing_account_id,
    relevant_contact_name as First_name,
    lower(relevant_contact_email) as Email_New,
    lower(email) as Email,
    relevant_contact_phone as Phone_2,
    multiIf(result == 'decline', '["Не готов общаться"]',
            status = 'no_response', '["Не дозвонились"]',
            null) as Dimensions,
    
    multiIf(CAST(Tags as String) like '%принять%', 'validated',
            CAST(Tags as String) like '%под вопросом%', 'tentative',
            'annulated') as Tags,
    if(instruction_required == 1, 'instruction_required', null) as Tags_New,
    'In Process' as Status,
    toDateTime(toInt64(assignment_approve_time / 1000)) as Last_communication_date,
    toString(ifNull(Comment, '')) || 
        '; Профиль партнера: ' || toString(ifNull(profile, '')) || 
        '; Цель заявки:  ' || toString(ifNull(goal, '')) || 
        '; Потенциальное потребление: ' || toString(ifNull(potential_consumption, '')) || 
        '; План привлечение клиентов: ' || toString(ifNull(plan, '')) ||  
        '; Вебсайт: ' || toString(ifNull(website, ''))  || 
        '; Нужна инструкция по созданию биллинг аккаунта: ' || toString(ifNull(instruction_required, 0)) ||
        '; preferred_communication: ' || toString(ifNull(preferred_communication, '')) ||
        '; questions_from_partner: ' || toString(ifNull(questions_from_partner, '')) as Notes
FROM "//home/cloud_analytics/call_center/output/var_output" as a
ANY LEFT JOIN (
    SELECT
        max(email) as email,
        max(billing_account_id) as billing_account_id,
        lead_id
    FROM "//home/cloud_analytics/kulaga/leads_cube"
    GROUP BY lead_id
) as b
ON CRM_Lead_ID == lead_id
FORMAT TabSeparatedWithNames
""".encode('utf-8')

update_df = lib.execute_query(req)

In [ ]:
def organise_new_old_values_as_string_in_array(x, name):
    add = []
    if not pd.isnull(x[name]):
        if name.lower() == 'email':
            add.append(lib.works_with_emails(x[name].strip()))
        else:
            add.append(x[name].strip())
    if not pd.isnull(x[name + "_New"]):
        if name.lower() == 'email':
            add.append(lib.works_with_emails(x[name + "_New"].strip()))
        else:
            add.append(x[name + "_New"].strip())
    if len(list(set(add))) > 0:
        return list(set(add))
    return np.nan

In [ ]:
update_df['Email'] = update_df.apply(lambda x: organise_new_old_values_as_string_in_array(x, "Email"), axis=1)
update_df['Billing_account_id'] = update_df.apply(
    lambda x: organise_new_old_values_as_string_in_array(x, "Billing_account_id"), axis=1)
update_df['Tags'] = update_df.apply(lambda x: organise_new_old_values_as_string_in_array(x, "Tags"), axis=1)

In [ ]:
arr_cols = ['Email', 'Billing_account_id', 'Tags', 'Dimensions']
for col in arr_cols:
    update_df[col] = update_df[col].astype(str)
    update_df[col] = update_df[col].apply(lambda x: x.replace("'", '"') if not pd.isnull(x) else np.nan)

In [ ]:
dropped = []
for col in update_df.columns:
    if "_New" in col:
        dropped.append(col)
update_df.drop(columns=dropped, inplace=True)

In [ ]:
old_df = lib.execute_query("""
SELECT
    DISTINCT 
    CRM_Lead_ID || '__' || toString(ifNull(Last_communication_date, '')) as key
FROM "//home/cloud_analytics/export/crm/update_call_center_leads/update_leads"
FORMAT TabSeparatedWithNames
""")

In [ ]:
update_df = update_df[~update_df[['CRM_Lead_ID', 'Last_communication_date']]\
          .apply(lambda x: x['CRM_Lead_ID'] + "__" + x['Last_communication_date'], axis=1)\
          .isin(old_df['key'])]

In [ ]:
update_df['Phone_2'] = update_df['Phone_2'].astype(str)

In [ ]:
update_df.replace("nan", np.nan, inplace=True)

In [ ]:
lib.save_table('update_leads', "//home/cloud_analytics/export/crm/update_call_center_leads",
               update_df, append=True)

In [ ]:
update_df